In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Isabel O.
[nltk_data]     Gallegos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Isabel O.
[nltk_data]     Gallegos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import json
import jsonlines
import pandas as pd #necessary to read .csv files
from sklearn.model_selection import train_test_split

import torch
import torchvision
import math
import numpy as np
from torch.utils.data import Dataset, DataLoader

#text-preprocessing
import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

In [6]:
#Load all of our data
#tldr_data and tosdr_data are dicts

tldr_data_file = open('./tldrlegal.json',)
tldr_data = json.load(tldr_data_file)

tosdr_data_file = open('./tosdr.json',)
tosdr_data = json.load(tosdr_data_file)

privacy_data = pd.read_csv('./privacy_policies_raw.csv')
privacy_data.head(10)

# billsum_path = "C:\\Users/Barry Cheung/Desktop/CS224N/billsum_v3"

# billsum_train_path = billsum_path + "/us_train_data_final_OFFICIAL.jsonl"
# billsum_train_file = open(billsum_path+"/us_train_data_final_OFFICIAL.jsonl")
# billsum_train_file = jsonlines.open(billsum_train_path)

# billsum_us_test_path = billsum_path + "/us_test_data_final_OFFICIAL.jsonl"
# billsum_ca_test_path = billsum_path + "/ca_test_data_final_OFFICIAL.jsonl"
# billsum_us_test_file = jsonlines.open(billsum_us_test_path)
# billsum_ca_test_file = jsonlines.open(billsum_ca_test_path)



In [8]:
def get_billsum_sum(billsum_file):
    data = pd.DataFrame(columns=['original_text', 'reference_summary'])
    for i, item in enumerate(billsum_file.iter()):
        current_item = item
        original_text_preprocess = preprocess(current_item['text'])
        reference_summary_preprocess = preprocess(current_item['summary'])
        data = data.append({'original_text':original_text_preprocess,
                            'reference_summary':reference_summary_preprocess}, ignore_index=True)
    return data

In [5]:
#punctuation_str = "\'!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~\'"
#print("corpora :", lemmatizer.lemmatize("liked"))
def remove_punctuation(input_string):
    res = re.sub(r'[^\w\s]', '', input_string)
    return res

#print(remove_punctuation("DAS: ABSCHELUCH!&*@# RITTER!!!!????,."))

def make_lowercase(input_string):
    return input_string.lower()

def remove_stopwords(input_string):
    tokens = word_tokenize(input_string)
    output_string = [w for w in tokens if not w in stop_words]
    return output_string

def lemmatization(input_list):
    output_string = ""
    #temp = input_string.split()
    for word in input_list:
        #print(word)
        lemmatized = lemmatizer.lemmatize(word)
        output_string += lemmatized + " "
    return output_string


def preprocess(input_string):
#     output = remove_punctuation(input_string)
    output = input_string
    output = make_lowercase(output)
    output = remove_stopwords(output)
    output = lemmatization(output)
    return output

def get_summary_from_json(json_file):
    data = pd.DataFrame(columns=['original_text', 'reference_summary'])
    #iterate through the text of the json files
    for i, item in enumerate(json_file):
        current_item = json_file[item]
        original_text_preprocess = preprocess(current_item['original_text'])
        reference_summary_preprocess = preprocess(current_item['reference_summary'])
        data = data.append({'original_text':original_text_preprocess,
                            'reference_summary':reference_summary_preprocess}, ignore_index=True)
    return data

In [6]:
get_summary_from_json(tldr_data).to_csv('./tldr/tldr.csv', index=False)
get_summary_from_json(tosdr_data).to_csv('./tosdr/tosdr.csv', index=False)

In [ ]:
get_billsum_sum(billsum_train_file).to_csv('us_billsum_train.csv', index=False)
get_billsum_sum(billsum_us_test_file).to_csv('ca_billsum_test.csv', index=False)
get_billsum_sum(billsum_ca_test_file).to_csv('us_billsum_test.csv', index=False)

In [2]:
# Test train split 70/15/15
tldr_df = pd.read_csv('./tldr/tldr.csv',)
tldr_train, tldr_test = train_test_split(tldr_df, test_size=0.3, random_state=0)
tldr_dev, tldr_test = train_test_split(tldr_test, test_size=0.5, random_state=0)

tldr_train.to_csv('./tldr/tldr_train.csv', index=False)
tldr_dev.to_csv('./tldr/tldr_dev.csv', index=False)
tldr_test.to_csv('./tldr/tldr_test.csv', index=False)

tosdr_df = pd.read_csv('./tosdr/tosdr.csv')
tosdr_train, tosdr_test = train_test_split(tosdr_df, test_size=0.3, random_state=0)
tosdr_dev, tosdr_test = train_test_split(tosdr_test, test_size=0.5, random_state=0)

tosdr_train.to_csv('./tosdr/tosdr_train.csv', index=False)
tosdr_dev.to_csv('./tosdr/tosdr_dev.csv', index=False)
tosdr_test.to_csv('./tosdr/tosdr_test.csv', index=False)

# us_billsum_train_df = pd.read_csv("./us_billsum_train.csv")
# us_billsum_train, us_billsum_dev = train_test_split(us_billsum_train_df, test_size=0.2)
# us_billsum_train.to_csv('./us_billsum_train_split.csv')
# us_billsum_dev.to_csv('./us_billsum_train_dev.csv')



In [12]:
#Make a Dataset, then DataLoader out of the privacy dataset
class privacyDataset(Dataset):
    def __init__(self):
        #data loading
        xy = np.loadtxt('./privacy_policies_raw.csv', delimiter = ",", dtype ='str', usecols=(0, 1), encoding ="utf8")
        self.x = xy[:, 1:]#torch.from_numpy(xy[:, 1:])#convert to tensor
        self.y = xy[:, [0]]#torch.from_numpy(xy[:, [0]])
        self.n_samples = xy.shape[0]
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        #dataset[0]
    def __len__(self):
        return self.n_samples

dataset = privacyDataset()
first_data = dataset[0]
features, labels = first_data
print(features)
print(labels)
print(first_data)
print(dataset)
#why does this last 25 minutes
# dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers = 2)
# dataiterator = iter(dataloader)

# data = dataiter.next()
# features, labels = data
# print(features, labels)

['QouteText']
['Service']
(array(['QouteText'], dtype='<U1450'), array(['Service'], dtype='<U1450'))


In [20]:
class CustomDataset(Dataset):
  def __init__(self,file_name):
    df=pd.read_csv(file_name)

    x=df['original_text'].values
    y=df['reference_summary'].values

    self.x=x
    self.y=y

  def __len__(self):
    return len(self.y)
  
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]

In [22]:
custom_dataset=CustomDataset('./tosdr/tosdr_train.csv')
train_loader=DataLoader(custom_dataset,batch_size=10,shuffle=False)
print(train_loader)